In [121]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xlwings as xw

def european_to_float(value):
    """
    Convert a European formatted string to a float.
    E.g., "30.416.600" -> 30416600.00
        "30.416.600,99" -> 30416600.99
    """
    if isinstance(value, str):
        value = value.replace('.', '').replace(',', '.')
    try:
        return float(value)
    except ValueError:
        return None

def get_security_params(infolist):
    disclosureIndex = infolist[0]

    # Step 1: Fetch HTML content
    url = f"https://www.kap.org.tr/tr/Bildirim/{disclosureIndex}"
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
    else:
        raise Exception(f"Failed to fetch webpage: {response.status_code}")

    # Step 2: Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Step 3: Extract several params
    paramdict = {}
    for label in ["ISIN Kodu", "Vade Tarihi", "Döviz Cinsi", "İhraç Fiyatı", "Faiz Oranı - Yıllık Basit (%)", "Satışı Gerçekleştirilen Nominal Tutar", "Satışa Başlanma Tarihi", "Kupon Sayısı"]:
        param = None
        for row in soup.find_all("tr"):
            label_div = row.find("div", class_="bold font14")
            if label_div and label in label_div.text:
                param = row.find("div", class_="gwt-HTML control-label lineheight-32px").text.strip()
                paramdict[label] = param
                break
        if param is None:  # If the label isn't found, set None
            paramdict[label] = param
    print(paramdict)

    # Extract the table rows manually
    table = soup.find_all("table")[5]
    rows = table.find_all("tr")

    # Extract the table headers
    headers = [header.text.strip() for header in rows[0].find_all("td")]
    
    # Extract the table data
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all("td")])

    # Create the DataFrame
    df = pd.DataFrame(data, columns=headers)
    df["Ödeme Tutarı"] = df["Ödeme Tutarı"].apply(european_to_float)
    df["Faiz Oranı - Dönemsel (%)"] = df["Faiz Oranı - Dönemsel (%)"].apply(european_to_float)
    df["COUPON_DATE"] = pd.to_datetime(df["Ödeme Tarihi"], format="%d.%m.%Y")
    df["ISIN_CODE"] = paramdict["ISIN Kodu"]
    
    # Set coupon rate depending if FRN or not
    if paramdict["Faiz Oranı - Yıllık Basit (%)"] == None:
        coupon = european_to_float(df["Faiz Oranı - Yıllık Basit (%)"][0])
    else:
        coupon = european_to_float(paramdict["Faiz Oranı - Yıllık Basit (%)"])

    df_security_coupon = df.loc[:, ["COUPON_DATE", "Faiz Oranı - Dönemsel (%)"]].dropna()
    df_security_coupon.columns = ["COUPON_DATE", "COUPON_RATE"]
    df_security_coupon.set_index("COUPON_DATE", inplace=True)

    # Security Sheet
    # Basis
    if paramdict["Döviz Cinsi"] == "TRY":
        basis = "ACTL365"
    elif paramdict["Döviz Cinsi"] == "EUR":
        basis = "EU30360"
    else:
        basis = "US30360"

    fis_dict = {
        "ISIN_CODE": paramdict["ISIN Kodu"],
        "INSTRUMENT_TYPE": None,  # instrument type is assigned below
        "MATURITY_DATE": pd.to_datetime(paramdict["Vade Tarihi"], format="%d.%m.%Y"),
        "CURRENCY": paramdict["Döviz Cinsi"],
        "FREQUENCY": int(paramdict["Kupon Sayısı"]),
        "COUPON": coupon,
        "SPREAD": 0,  # hard-coded, fix later
        "ISSUER_CODE": infolist[2],  # second element of input list is issuer code
        "ISSUE_INDEX": 0,  # hard-coded, fix later
        "ISSUE_DATE": pd.to_datetime(paramdict["Satışa Başlanma Tarihi"], format="%d.%m.%Y"),
        "DAY_YEAR_BASIS": basis,
        "ISSUE_PRICE": european_to_float(paramdict["İhraç Fiyatı"]) * 100,
        "totalIssuedAmount": european_to_float(paramdict["Satışı Gerçekleştirilen Nominal Tutar"]),
        "securityType": None,  # hard-coded, fix later
        "fundUser": None  # hard-coded, fix later
    }

    # Instrument Type
    if infolist[1]:
        if fis_dict["FREQUENCY"] == 0:
            fis_dict["INSTRUMENT_TYPE"] = "CORP_SUKUK_DISCOUNTED"
        else:
            if fis_dict["SPREAD"] == 0:
                fis_dict["INSTRUMENT_TYPE"] = "CORP_SUKUK_FIXED_COUPON"
            else:
                fis_dict["INSTRUMENT_TYPE"] = "CORP_SUKUK_FLOATING"
    else:
        if fis_dict["CURRENCY"] == "TRY":
            if fis_dict["FREQUENCY"] == 0:
                fis_dict["INSTRUMENT_TYPE"] = "CORP_DISCOUNTED"
            else:
                if fis_dict["ISSUE_INDEX"] == 0:
                    if fis_dict["SPREAD"] == 0:  # there are actually 0 spread FRN's, fix this later
                        fis_dict["INSTRUMENT_TYPE"] = "CORP_FIXED_COUPON"
                    else:
                        fis_dict["INSTRUMENT_TYPE"] = "CORP_FLOATING"
                else:
                    fis_dict["INSTRUMENT_TYPE"] = "TÜFEX"
        else:
            fis_dict["INSTRUMENT_TYPE"] = "EUROBOND"

    # Generate security dataframe from fis_dict
    df_security = pd.Series(fis_dict)

    return df_security, df_security_coupon

infolist = ["1282416", False, "FBB"]
df1, df2 = get_security_params(infolist)
df1

def parse_disclosures():
    paramdict = {}
    # Retrieve the data from the API
    url = "https://www.kap.org.tr/tr/api/disclosures"
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        disclosures = json.loads(response.text)
        
        # Extract 'basic' and 'detail' data into separate lists of dictionaries
        basic_data = [disclosure['basic'] for disclosure in disclosures]
        # detail_data = [disclosure['detail'] for disclosure in disclosures]
    else:
        print("Failed to retrieve data from the API")
        
    for disc in basic_data:
        if disc["title"] in ["Pay Dışında Sermaye Piyasası Aracı İşlemlerine İlişkin Bildirim (Faiz İçeren)", "Pay Dışında Sermaye Piyasası Aracı İşlemlerine İlişkin Bildirim (Faizsiz)"]:
            paramdict[disc["disclosureIndex"]] = [disc["stockCodes"]]
    
    print("type:", type(basic_data))
    print("len:", len(basic_data))
    
    return paramdict

{'ISIN Kodu': 'TRPFB2F52416', 'Vade Tarihi': '07.05.2024', 'Döviz Cinsi': 'TRY', 'İhraç Fiyatı': '1', 'Faiz Oranı - Yıllık Basit (%)': None, 'Satışı Gerçekleştirilen Nominal Tutar': '35.000.000', 'Satışa Başlanma Tarihi': '08.08.2023', 'Kupon Sayısı': '1'}


In [122]:
def parse_disclosures():
    paramlist = []
    # Retrieve the data from the API
    url = "https://www.kap.org.tr/tr/api/disclosures"
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        disclosures = json.loads(response.text)
        
        # Extract 'basic' data into list of dictionaries
        basic_data = [disclosure['basic'] for disclosure in disclosures]
        # detail_data = [disclosure['detail'] for disclosure in disclosures]
    else:
        print("Failed to retrieve data from the API")
        
    for disc in basic_data:
        if disc["title"] == "Pay Dışında Sermaye Piyasası Aracı İşlemlerine İlişkin Bildirim (Faiz İçeren)":
            paramlist.append([disc["disclosureIndex"], False, disc["stockCodes"].split(',')[0].strip()])  # [sukuk flag, issuer code]
        elif disc["title"] == "Pay Dışında Sermaye Piyasası Aracı İşlemlerine İlişkin Bildirim (Faizsiz)":
            paramlist.append([disc["disclosureIndex"], True, disc["stockCodes"].split(',')[0].strip()])  # [sukuk flag, issuer code]
    
    # print("type:", type(basic_data))
    # print("len:", len(basic_data))
    
    return paramlist

In [123]:
parse_disclosures()

[[1282451, False, 'DARDL'],
 [1282427, True, 'BRKT'],
 [1282416, False, 'FBB'],
 [1282415, False, 'FBB'],
 [1282411, True, 'EKTVK'],
 [1282403, True, 'KTKVK']]

In [124]:
infolist = ["1281578", False, "FBB"]
df1, df2 = get_security_params(infolist)
df1

{'ISIN Kodu': 'TRSKORT52525', 'Vade Tarihi': '22.05.2025', 'Döviz Cinsi': 'TRY', 'İhraç Fiyatı': '1', 'Faiz Oranı - Yıllık Basit (%)': '61', 'Satışı Gerçekleştirilen Nominal Tutar': '200.000.000', 'Satışa Başlanma Tarihi': '02.05.2024', 'Kupon Sayısı': '4'}


ISIN_CODE                   TRSKORT52525
INSTRUMENT_TYPE        CORP_FIXED_COUPON
MATURITY_DATE        2025-05-22 00:00:00
CURRENCY                             TRY
FREQUENCY                              4
COUPON                              61.0
SPREAD                                 0
ISSUER_CODE                          FBB
ISSUE_INDEX                            0
ISSUE_DATE           2024-05-02 00:00:00
DAY_YEAR_BASIS                   ACTL365
ISSUE_PRICE                        100.0
totalIssuedAmount            200000000.0
securityType                        None
fundUser                            None
dtype: object

In [125]:
df2

,COUPON_RATE
COUPON_DATE,
2024-08-05,15.2083
2024-11-04,15.2083
2025-02-03,15.2083
2025-05-22,18.0494


In [126]:
infolist = ["1282415", False, "FBB"]
df1, df2 = get_security_params(infolist)
df1

{'ISIN Kodu': 'TRPFB1F52418', 'Vade Tarihi': '07.05.2024', 'Döviz Cinsi': 'TRY', 'İhraç Fiyatı': '1', 'Faiz Oranı - Yıllık Basit (%)': None, 'Satışı Gerçekleştirilen Nominal Tutar': '80.000.000', 'Satışa Başlanma Tarihi': '08.08.2023', 'Kupon Sayısı': '1'}


ISIN_CODE                   TRPFB1F52418
INSTRUMENT_TYPE        CORP_FIXED_COUPON
MATURITY_DATE        2024-05-07 00:00:00
CURRENCY                             TRY
FREQUENCY                              1
COUPON                              94.7
SPREAD                                 0
ISSUER_CODE                          FBB
ISSUE_INDEX                            0
ISSUE_DATE           2023-08-08 00:00:00
DAY_YEAR_BASIS                   ACTL365
ISSUE_PRICE                        100.0
totalIssuedAmount             80000000.0
securityType                        None
fundUser                            None
dtype: object

In [127]:
df2

,COUPON_RATE
COUPON_DATE,
2024-05-07,70.5708
